
### Import Libraries

In [11]:
import numpy as np
import random
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

### Dataset

In [12]:
california_housing = fetch_california_housing()
california_housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

### Train-Test Split

In [13]:
def train_test_split(X,y,test=0.2):

    train_size = X.shape[0]-int(X.shape[0]*test)
    permutations=np.random.permutation(X.shape[0])

    X=X[permutations]
    y=y[permutations]

    X_train = X[:train_size]
    y_train = y[:train_size]
    X_test = X[train_size:]
    y_test = y[train_size:]

    return X_train, X_test, y_train, y_test

In [14]:
scaler=StandardScaler()
X_normalized=scaler.fit_transform(california_housing["data"])
X_train, X_test, y_train, y_test = train_test_split(X_normalized,california_housing["target"].reshape(-1,1),0.2)

In [15]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(16512, 8)
(4128, 8)
(16512, 1)
(4128, 1)


### Neural Network

In [16]:
import numpy as np

class NeuralNetwork:
    def __init__(self, num_inputs=8,hidden_layers=10, learning_rate=0.001):
        self.lr = learning_rate
        self.w1 = np.random.randn(hidden_layers, num_inputs)
        self.b1 = np.zeros((hidden_layers,1))
        self.w2 = np.random.randn(1,hidden_layers)
        self.b2 = np.random.randn(1,1)
    
    def model(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    
    def ReLU_foward(self, x):
        return np.maximum(0, x),(x>0).astype(int)

    
    def affine_forward(self,x,w,b):
        out=np.dot(x,w.T)+np.repeat(b.T,1,axis=0)
        return out,w,x,np.ones(x.shape[0])

    def affine_backward(self,ud,ldx,ldw,ldb):
        db=np.sum(ud,axis=0)
        dw=np.dot(ud.T,ldw)
        dx=np.dot(ud,ldx)

        return dx,db,dw
    
    def loss_function(self,predicted,actual):
        loss=1/2*np.sum((predicted-actual)**2)
        dloss=predicted-actual
        return loss,dloss

    def gradient_descent(self):

        out1,ldx1,ldw1,ldb1=self.affine_forward(self.X_train,self.w1,self.b1)
        a_out1,lda_out1=self.ReLU_foward(out1)
        out2,ldx2,ldw2,ldb2=self.affine_forward(a_out1,self.w2,self.b2)

        self.Loss,dloss=self.loss_function(out2,self.y_train)
 
        dx2,db2,dw2=self.affine_backward(dloss,ldx2,ldw2,ldb2)
        da_out1= dx2 * (lda_out1>0)
        dx1,db1,dw1=self.affine_backward(da_out1,ldx1,ldw1,ldb1)


        db1=db1.reshape(-1,1)
        db2=db2.reshape(-1,1)

        self.w2 -= self.lr*dw2
        self.b2 -= self.lr*db2
        self.w1 -= self.lr*dw1
        self.b1 -= self.lr*db1

        return out2
    
    def output(self):
        out1,ldx1,ldw1,ldb1=self.affine_forward(self.X_train,self.w1,self.b1)
        a_out1,lda_out1=self.ReLU_foward(out1)
        out2,ldx2,ldw2,ldb2=self.affine_forward(a_out1,self.w2,self.b2)
        return out2

    def loss(self):
        return self.Loss

### Training Neural Network

#### Setting Parameters

In [17]:
num_inputs=8
hiddern_layers=64
learning_rate=0.0001
epochs=30
batch_size=5

In [18]:
NN=NeuralNetwork(num_inputs=num_inputs,hidden_layers=hiddern_layers,learning_rate=learning_rate)

#### Training

In [19]:
for epoch in range(0,epochs):
    for batch in range(0,X_train.shape[0],batch_size):
        NN.model(X_train[batch].reshape(1,-1),y_train[batch].reshape(1,-1))

        output=NN.gradient_descent()
        
        if batch%1000==0:
            print("Predicted : ",output,"   Actual : ",y_train[batch].reshape(1,-1))
            print("Loss : ",NN.loss())
            print()

Predicted :  [[-23.29186732]]    Actual :  [[1.115]]
Loss :  297.84758627821094

Predicted :  [[-3.75971588]]    Actual :  [[2.594]]
Loss :  20.18485271934536

Predicted :  [[1.68428854]]    Actual :  [[1.998]]
Loss :  0.0492074393404457

Predicted :  [[2.89685846]]    Actual :  [[3.209]]
Loss :  0.048716170571404184

Predicted :  [[1.16996235]]    Actual :  [[0.79]]
Loss :  0.07218569251119021

Predicted :  [[-0.68884789]]    Actual :  [[1.554]]
Loss :  2.5151833329651248

Predicted :  [[2.77051212]]    Actual :  [[2.719]]
Loss :  0.0013267495062873724

Predicted :  [[9.63216539]]    Actual :  [[0.55]]
Loss :  41.242864074819664

Predicted :  [[3.55264096]]    Actual :  [[1.906]]
Loss :  1.355713227465534

Predicted :  [[2.47537909]]    Actual :  [[3.]]
Loss :  0.13761354828791325



Predicted :  [[3.50227846]]    Actual :  [[0.647]]
Loss :  4.076307552223572

Predicted :  [[2.51972921]]    Actual :  [[2.706]]
Loss :  0.017348404360740182

Predicted :  [[4.17824327]]    Actual :  [[0.696]]
Loss :  6.063009097753391

Predicted :  [[1.20226583]]    Actual :  [[1.244]]
Loss :  0.0008708703570258904

Predicted :  [[0.3493278]]    Actual :  [[1.031]]
Loss :  0.23233849362144599

Predicted :  [[5.18575916]]    Actual :  [[3.813]]
Loss :  0.9422338565870141

Predicted :  [[3.10239876]]    Actual :  [[4.286]]
Loss :  0.7004559459735947

Predicted :  [[-4.78087624]]    Actual :  [[1.115]]
Loss :  17.380678290996922

Predicted :  [[-0.05285961]]    Actual :  [[2.594]]
Loss :  3.5029329014907153

Predicted :  [[4.78723342]]    Actual :  [[1.998]]
Loss :  3.8899115396314867

Predicted :  [[3.83585278]]    Actual :  [[3.209]]
Loss :  0.1964722068861461

Predicted :  [[0.83790898]]    Actual :  [[0.79]]
Loss :  0.0011476354135796245

Predicted :  [[0.30145496]]    Actual :  [[1.

### Testing Neural Network

#### Predicting Values

In [20]:
loss=[]
for batch in range(0,X_test.shape[0],batch_size):
    NN.model(X_test[batch].reshape(1,-1),y_test[batch].reshape(1,-1))
    output=NN.output()

    print("Predicted : ",output,"   Actual : ",y_train[batch].reshape(1,-1))

Predicted :  [[0.96031641]]    Actual :  [[1.115]]
Predicted :  [[1.42931806]]    Actual :  [[2.181]]
Predicted :  [[2.20919777]]    Actual :  [[5.00001]]
Predicted :  [[0.91491786]]    Actual :  [[2.125]]
Predicted :  [[1.99489523]]    Actual :  [[2.751]]
Predicted :  [[1.16462666]]    Actual :  [[3.429]]
Predicted :  [[2.85483861]]    Actual :  [[1.559]]
Predicted :  [[1.0341071]]    Actual :  [[5.00001]]
Predicted :  [[1.30998051]]    Actual :  [[2.263]]
Predicted :  [[1.88166742]]    Actual :  [[2.375]]
Predicted :  [[2.98833145]]    Actual :  [[1.178]]
Predicted :  [[3.05441356]]    Actual :  [[1.132]]
Predicted :  [[2.58167079]]    Actual :  [[0.736]]
Predicted :  [[3.61482044]]    Actual :  [[2.261]]
Predicted :  [[2.90917101]]    Actual :  [[2.279]]
Predicted :  [[2.1308274]]    Actual :  [[3.427]]
Predicted :  [[2.56907166]]    Actual :  [[1.752]]
Predicted :  [[0.81571834]]    Actual :  [[1.709]]
Predicted :  [[2.2791382]]    Actual :  [[1.583]]
Predicted :  [[2.22955358]]   